[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain/blob/master/docs/docs/use_cases/chatbots.ipynb)

# Quickstart

## Overview

We'll go over an example of how to design and implement an LLM-powered chatbot. Here are a few of the high-level components we'll be working with:

- `Chat Models`. The chatbot interface is based around messages rather than raw text, and therefore is best suited to Chat Models rather than text LLMs. See [here](/docs/integrations/chat) for a list of chat model integrations and [here](/docs/modules/model_io/chat) for documentation on the chat model interface in LangChain. You can use `LLMs` (see [here](/docs/modules/model_io/llms)) for chatbots as well, but chat models have a more conversational tone and natively support a message interface.
- `Prompt Templates`, which simplify the process of assembling prompts that combine default messages, user input, chat history, and (optionally) additional retrieved context.
- `Chat History`, which allows a chatbot to "remember" past interactions and take them into account when responding to followup questions. [See here](/docs/modules/memory/chat_messages/) for more information.
- `Retrievers` (optional), which are useful if you want to build a chatbot that can use domain-specific, up-to-date knowledge as context to augment its responses. [See here](/docs/modules/data_connection/retrievers) for in-depth documentation on retrieval systems.

We'll cover how to fit the above components together to create a powerful conversational chatbot.

## Quickstart

To start, let's install some dependencies and set the required credentials:

In [1]:
%pip install --upgrade --quiet langchain langchain-openai

# Set env var OPENAI_API_KEY or load from a .env file:
import dotenv

dotenv.load_dotenv()

You should consider upgrading via the '/Users/jacoblee/.pyenv/versions/3.10.5/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


True

Let's initialize the chat model which will serve as the chatbot's brain:

In [2]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo-1106")

If we invoke our chat model, the output is an `AIMessage`:

In [3]:
from langchain_core.messages import HumanMessage

chat.invoke(
    [
        HumanMessage(
            content="Translate this sentence from English to French: I love programming."
        )
    ]
)

AIMessage(content="J'adore la programmation.")

The model on its own does not have any concept of state. For example, if you ask a followup question:

In [4]:
chat.invoke([HumanMessage(content="What did you just say?")])

AIMessage(content='I said "What did you just say?"')

We can see that it doesn't take the previous conversation turn into context, and cannot answer the question.

To get around this, we need to pass the entire conversation history into the model. Let's see what happens when we do that:

In [5]:
from langchain_core.messages import AIMessage

chat.invoke(
    [
        HumanMessage(
            content="Translate this sentence from English to French: I love programming."
        ),
        AIMessage(content="J'adore la programmation."),
        HumanMessage(content="What did you just say?"),
    ]
)

AIMessage(content='I said "I love programming" in French.')

And now we can see that we get a good response!

This is the basic idea underpinning a chatbot's ability to interact conversationally.

## Prompt templates

Let's define a prompt template to make formatting inputs to the chat model a bit easier and create a chain by piping it into the model:

In [6]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
    ]
)

chain = prompt | chat

The `MessagesPlaceholder` above inserts chat messages passed into the chain's input as `chat_history` directly into the prompt. Then, we can invoke the chain like this:

In [7]:
chain.invoke(
    {
        "chat_history": [
            HumanMessage(
                content="Translate this sentence from English to French: I love programming."
            ),
            AIMessage(content="J'adore la programmation."),
        ],
        "input": "What did you just say?",
    }
)

AIMessage(content='I said "J\'adore la programmation," which means "I love programming" in French.')

## Message history

As a shortcut for managing the chat history, we can use a [`MessageHistory`](/docs/modules/memory/chat_messages/) class, which is responsible for saving and loading chat messages. There are many built-in message history integrations that persist messages to a variety of databases, but for this quickstart we'll use a in-memory, demo message history called `ChatMessageHistory`.

Here's an example of using it directly:

In [8]:
from langchain.memory import ChatMessageHistory

demo_ephemeral_chat_history = ChatMessageHistory()

demo_ephemeral_chat_history.add_user_message("hi!")

demo_ephemeral_chat_history.add_ai_message("whats up?")

demo_ephemeral_chat_history.messages

[HumanMessage(content='hi!'), AIMessage(content='whats up?')]

To automate this saving/loading process for our chains, we can wrap it in a `RunnableWithMessageHistory` class that automatically saves and loads messages from the chat history, updating it with a `HumanMessage` created from the input passed under `input_messages_key` and an `AIMessage` from the chain output.

The chat history is returned from a function because in real-world use-cases like web servers, one instance of a chain might handle multiple sessions at once. For this demo, we return the same chat history instance each time.

In [9]:
from langchain_core.runnables.history import RunnableWithMessageHistory

demo_ephemeral_chat_history_for_chain = ChatMessageHistory()

chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: demo_ephemeral_chat_history_for_chain,
    input_messages_key="input",
    history_messages_key="chat_history",
)

Let's invoke it and see what happens! Note that we need to pass a `session_id` - in real-world use-cases, we would return a different chat history object within the function depending on this passed parameter:

In [10]:
chain_with_message_history.invoke(
    {"input": "Translate this sentence from English to French: I love programming."},
    {"configurable": {"session_id": "unused"}},
)

AIMessage(content='The translation of "I love programming" in French is "J\'adore la programmation."')

In [11]:
chain_with_message_history.invoke(
    {"input": "What did I just ask you?"}, {"configurable": {"session_id": "unused"}}
)

AIMessage(content='You just asked me to translate the sentence "I love programming" from English to French.')

And now we have a basic chatbot!

While this `chain_with_message_history` chain can serve as a useful chatbot on its own with just the model's internal knowledge, it's common to introduce some form of `retrieval-augmented generation`, or RAG for short, over domain-specific knowledge to make our chatbot more focused. We'll cover this next.

## Retrievers

We can set up and use a `Retriever` to pull domain-specific knowledge for our chatbot. To show this, let's expand the simple chatbot we created above to be able to answer questions about LangSmith.

We'll use [the LangSmith documentation](https://docs.smith.langchain.com/overview) as source material. Note that this example will gloss over some of the specifics around parsing and storing a data source - you can see more [in-depth documentation on creating retrieval systems here](https://python.langchain.com/docs/use_cases/question_answering/).

Let's set up our retriever. First, we pull data from a webpage:

In [12]:
%pip install --upgrade --quiet tiktoken chromadb beautifulsoup4

You should consider upgrading via the '/Users/jacoblee/.pyenv/versions/3.10.5/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [13]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
data = loader.load()

Next, we split it into smaller chunks that the LLM's context window can handle and store it in a vector database:

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

Then we embed and store those chunks in a vector database:

In [15]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

And finally, let's create a retriever from our initialized vectorstore:

In [16]:
# k is the number of chunks to retrieve
retriever = vectorstore.as_retriever(k=4)

docs = retriever.invoke("how can langsmith help with testing?")

docs

[Document(page_content='You can also quickly edit examples and add them to datasets to expand the surface area of your evaluation sets or to fine-tune a model for improved quality or reduced costs.Monitoring\u200bAfter all this, your app might finally ready to go in production. LangSmith can also be used to monitor your application in much the same way that you used for debugging. You can log all traces, visualize latency and token usage statistics, and troubleshoot specific issues as they arise. Each run can also be', metadata={'description': 'Building reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'LangSmith Overview and User Guide | 🦜️🛠️ LangSmith'}),
 Document(page_content='inputs, and see what happens. At s

We can see that invoking the retriever above results in some parts of the LangSmith docs that contain information about testing that our chatbot can use as context when answering questions.

Let's modify our previous prompt to accept documents as context. We'll use a `create_stuff_documents_chain` helper function to "stuff" all of the input documents into the prompt, which also conveniently handles formatting:

In [17]:
from langchain.chains.combine_documents import create_stuff_documents_chain

chat = ChatOpenAI(model="gpt-3.5-turbo-1106")

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user's questions based on the below context:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
    ]
)

document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

We can invoke this `document_chain` with the raw documents we retrieved above:

In [18]:
document_chain.invoke(
    {
        "input": "how can langsmith help with testing?",
        "chat_history": [],
        "context": docs,
    }
)

'LangSmith can help with testing in several ways. It allows you to quickly edit examples and add them to datasets, expanding the surface area of your evaluation sets. This enables you to fine-tune a model for improved quality or reduced costs. Additionally, LangSmith simplifies the construction of small datasets by hand, providing a more rigorous approach to testing changes in your application. Furthermore, it facilitates monitoring your application by logging all traces, visualizing latency and token usage statistics, and troubleshooting specific issues as they arise.'

Awesome! We see an answer synthesized from information in the input documents.

Next, let's integrate our retriever into the chain. We'll use the `create_retrieval_chain` method, which expects a retriever and a chain that takes `context`, `input`, and `chat_history` as input parameters. It returns all of the intermediate values in its output:

In [19]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, document_chain)

retrieval_chain.invoke(
    {
        "input": "how can langsmith help with testing?",
        "chat_history": [],
    }
)

{'input': 'how can langsmith help with testing?',
 'chat_history': [],
 'context': [Document(page_content='You can also quickly edit examples and add them to datasets to expand the surface area of your evaluation sets or to fine-tune a model for improved quality or reduced costs.Monitoring\u200bAfter all this, your app might finally ready to go in production. LangSmith can also be used to monitor your application in much the same way that you used for debugging. You can log all traces, visualize latency and token usage statistics, and troubleshoot specific issues as they arise. Each run can also be', metadata={'description': 'Building reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'LangSmith Overview and User Gu

Then, we can wrap that `retrieval_chain` in the same `RunnableWithMessageHistory` as before. Note that because our retrieval chain now has multiple outputs, we must specify the which part of the output we want to save as chat history using `output_messages_key`:

In [20]:
demo_ephemeral_chat_history_for_chain = ChatMessageHistory()

retrieval_chain_with_message_history = RunnableWithMessageHistory(
    retrieval_chain,
    lambda session_id: demo_ephemeral_chat_history_for_chain,
    input_messages_key="input",
    output_messages_key="answer",
    history_messages_key="chat_history",
)

And now our chatbot chain can answer questions about LangSmith!

In [21]:
retrieval_chain_with_message_history.invoke(
    {
        "input": "how can langsmith help with testing?",
    },
    {"configurable": {"session_id": "unused"}},
)

{'input': 'how can langsmith help with testing?',
 'chat_history': [],
 'context': [Document(page_content='You can also quickly edit examples and add them to datasets to expand the surface area of your evaluation sets or to fine-tune a model for improved quality or reduced costs.Monitoring\u200bAfter all this, your app might finally ready to go in production. LangSmith can also be used to monitor your application in much the same way that you used for debugging. You can log all traces, visualize latency and token usage statistics, and troubleshoot specific issues as they arise. Each run can also be', metadata={'description': 'Building reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'LangSmith Overview and User Gu

In [22]:
retrieval_chain_with_message_history.invoke(
    {
        "input": "tell me more about that!",
    },
    {"configurable": {"session_id": "unused"}},
)

{'input': 'tell me more about that!',
 'chat_history': [HumanMessage(content='how can langsmith help with testing?'),
  AIMessage(content='LangSmith can assist with testing by simplifying the construction of datasets for evaluation sets or fine-tuning models. It allows you to quickly edit examples and add them to datasets to expand the surface area of your evaluation sets. Additionally, LangSmith enables monitoring of your application by logging all traces, visualizing latency and token usage statistics, and troubleshooting specific issues as they arise. This monitoring capability can help in testing changes rigorously and ensuring that your application is performing well before going into production.')],
 'context': [Document(page_content='however, there is still no complete substitute for human review to get the utmost quality and reliability from your application.', metadata={'description': 'Building reliable LLM applications can be challenging. LangChain simplifies the initial setu

Nice! Our chatbot can now answer domain-specific questions in a conversational way.

## Query transformation

There's more optimization we'll cover here - in the above example, when we asked a followup question, `tell me more about that!`, you might notice that the retrieved docs don't directly include information about testing. This is because we're passing `tell me more about that!` verbatim as a query to the retriever. The output in the retrieval chain is still okay because the document chain retrieval chain can generate an answer based on the chat history, but we could be retrieving more rich and informative documents:

In [23]:
retriever.invoke("how can langsmith help with testing?")

[Document(page_content='You can also quickly edit examples and add them to datasets to expand the surface area of your evaluation sets or to fine-tune a model for improved quality or reduced costs.Monitoring\u200bAfter all this, your app might finally ready to go in production. LangSmith can also be used to monitor your application in much the same way that you used for debugging. You can log all traces, visualize latency and token usage statistics, and troubleshoot specific issues as they arise. Each run can also be', metadata={'description': 'Building reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'LangSmith Overview and User Guide | 🦜️🛠️ LangSmith'}),
 Document(page_content='inputs, and see what happens. At s

In [24]:
retriever.invoke("tell me more about that!")

[Document(page_content='however, there is still no complete substitute for human review to get the utmost quality and reliability from your application.', metadata={'description': 'Building reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'LangSmith Overview and User Guide | 🦜️🛠️ LangSmith'}),
 Document(page_content='You can also quickly edit examples and add them to datasets to expand the surface area of your evaluation sets or to fine-tune a model for improved quality or reduced costs.Monitoring\u200bAfter all this, your app might finally ready to go in production. LangSmith can also be used to monitor your application in much the same way that you used for debugging. You can log all traces, visualize latency an

To get around this common problem, let's add a `query transformation` step that removes references from the input. We'll wrap our old retriever using the `create_history_aware_retriever` method as follows:

In [25]:
from langchain.chains import create_history_aware_retriever

# We need a prompt that we can pass into an LLM to generate a transformed search query

chat = ChatOpenAI(model="gpt-3.5-turbo-1106")

query_transform_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        (
            "user",
            "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation",
        ),
    ]
)

history_aware_retriever_chain = create_history_aware_retriever(
    chat, retriever, query_transform_prompt
)

Now let's recreate our earlier chain with this new retriever chain:

In [26]:
question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user's questions based on the below context:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
    ]
)

document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

retrieval_chain = create_retrieval_chain(history_aware_retriever_chain, document_chain)

demo_ephemeral_chat_history_for_chain = ChatMessageHistory()

conversational_retrieval_chain = RunnableWithMessageHistory(
    retrieval_chain,
    lambda session_id: demo_ephemeral_chat_history_for_chain,
    input_messages_key="input",
    output_messages_key="answer",
    history_messages_key="chat_history",
)

And finally, let's invoke it!

In [27]:
conversational_retrieval_chain.invoke(
    {"input": "how can langsmith help with testing?"},
    {"configurable": {"session_id": "unused"}},
)

{'input': 'how can langsmith help with testing?',
 'chat_history': [],
 'context': [Document(page_content='You can also quickly edit examples and add them to datasets to expand the surface area of your evaluation sets or to fine-tune a model for improved quality or reduced costs.Monitoring\u200bAfter all this, your app might finally ready to go in production. LangSmith can also be used to monitor your application in much the same way that you used for debugging. You can log all traces, visualize latency and token usage statistics, and troubleshoot specific issues as they arise. Each run can also be', metadata={'description': 'Building reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'LangSmith Overview and User Gu

In [28]:
conversational_retrieval_chain.invoke(
    {"input": "tell me more about that!"}, {"configurable": {"session_id": "unused"}}
)

{'input': 'tell me more about that!',
 'chat_history': [HumanMessage(content='how can langsmith help with testing?'),
  AIMessage(content='LangSmith can help with testing by simplifying the process of constructing and utilizing datasets for testing and evaluation. It allows you to quickly edit examples and add them to datasets, which expands the surface area of your evaluation sets. Additionally, LangSmith enables you to fine-tune a model for improved quality or reduced costs. It also provides monitoring capabilities, allowing you to log all traces, visualize latency and token usage statistics, and troubleshoot specific issues as they arise during testing. Overall, LangSmith streamlines the testing process and enhances the rigor of testing changes for improved reliability.')],
 'context': [Document(page_content='LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', metadata={'description': 'Building reliable LLM applications can be challenging. LangChain simplifies the initial setup, bu

// TODO: add LangSmith traces illustrating transformed queries

## Next steps

You now know how to build a conversational chatbot that can integrate past messages and domain-specific knowledge into its generations. There are many other optimizations you can make around this - check out the following pages for more information:

- Memory management: This includes trimming, summarizing, or otherwise modifying long conversations to keep your bot focused.
- Different interfaces: For when you want finer-grained control over inputs and outputs
- Retrieval: A deeper dive into using different types of retrieval with your chatbot
- Tool usage: How to allows your chatbots to use tools that interact with other APIs and systems.